# Laboratory 2

## Task 1
Create two classes of random points in 2D plain. Each class should contain 1000 points from the bivariate normal distribution:
- the first with mean $\mu_1=[0, 1.5]$ and covariance matrix $\Sigma_1=\begin{bmatrix}1/3& 1/6\\1/6& 1/3\end{bmatrix}$
- the second with mean $\mu_2=[2,0]$ and covariance matrix $\Sigma_2=\begin{bmatrix}2/3& 1/3\\1/3& 2/3\end{bmatrix}$

Next generate corresponding target labels: 0 for the first class and 1 for the second class and visualize two classes:
<img src="../lectures/Lecture 2-20250316/1.png" alt="Two classes"/>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)
num_points = 1000
mean1 = [0, 1.5]
cov1 = [[1/3, 1/6], [1/6, 1/3]]
mean2 = [2, 0]
cov2 = [[2/3, 1/3], [1/3, 2/3]]
points1 = np.random.multivariate_normal(mean1, cov1, num_points)
points2 = np.random.multivariate_normal(mean2, cov2, num_points)
targets1 = np.zeros(num_points)
targets2 = np.ones(num_points)

plt.figure(figsize=(8, 6))
plt.scatter(points1[:, 0], points1[:, 1], color='blue')
plt.scatter(points2[:, 0], points2[:, 1], color='yellow')
plt.show()

## Task 2
Create the linear classifier of the form $prediction = W \cdot input + b$, where $W$ is tensorflow variable with initial value being random uniform vector of shape (2,1) and $b$ is tensorflow variable with initial value equal 0. Using tensor operations define the forward pass function returning for input: $W \cdot input + b$.

In [ ]:
import tensorflow as tf

W = tf.Variable(tf.random.uniform(shape=(2, 1), minval=-0.1, maxval=0.1), dtype=tf.float32)
b = tf.Variable(0.0, dtype=tf.float32)

def forward(inputs):
    return tf.matmul(inputs, W) + b

## Task 3
For input: targets and predictions, define the mean squared error loss function using tensor operations.

In [ ]:
def loss(targets, predictions):
    return tf.reduce_mean(tf.square(targets - predictions))

## Task 4
For learning rate 0.1 define the training step function. The function for arguments: inputs, targets, should:
- do forward pass, inside a gradient tape scope
- retrieve the gradient of the loss with regard to weights 
- update the weights
- return loss

In [ ]:
learning_rate = 0.1
optimizer = tf.optimizers.SGD(learning_rate)

def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = forward(inputs)
        current_loss = loss(targets, predictions)
    gradients = tape.gradient(current_loss, [W, b])

    optimizer.apply_gradients(zip(gradients, [W, b]))

    return current_loss

## Task 5
Perform the training step function 50 times.

In [ ]:
# Połączenie danych i etykiet
inputs = np.vstack((points1, points2)).astype(np.float32)
targets = np.hstack((np.zeros(1000), np.ones(1000))).astype(np.float32).reshape(-1, 1)

# Konwersja na tensory TensorFlow
inputs = tf.convert_to_tensor(inputs)
targets = tf.convert_to_tensor(targets)

for i in range(50):
    current_loss = train_step(inputs, targets)
    print(f"Loss at step {i}: {current_loss.numpy()}")



## Task 6
Do predictions for the model: a given input point will be classified as “0” if its prediction value is below 0.5, and as “1” if it is above 0.5. Visualize model predictions:
<img src="../lectures/Lecture 2-20250316/2.png" alt="Model predictions"/>

In [ ]:
import numpy as np
# Predykcja dla danych testowych
predictions = forward(inputs).numpy()  # Forward pass
predicted_classes = (predictions > 0.5).astype(int)  # Klasyfikacja (0 lub 1)

# Wizualizacja predykcji w stylu "plt.scatter"
plt.figure(figsize=(8, 6))

# Rysowanie klasy 0 (fioletowy)
points1 = inputs.numpy()[predicted_classes.flatten() == 0]
plt.scatter(points1[:, 0], points1[:, 1], color='blue')

# Rysowanie klasy 1 (żółty)
points2 = inputs.numpy()[predicted_classes.flatten() == 1]
plt.scatter(points2[:, 0], points2[:, 1], color='yellow')

plt.show()



# second class
plt.show()

## Task 7
Add to the above plot  a classifying line in the 2D plane
$$y = -\frac{w_1}{w_2}x + \frac{0.5-b}{w_2}$$ 
<img src="../lectures/Lecture 2-20250316/3.png" alt="Classifying lines with predicted targets"/>
Next add the classifying line to plot of samples with original targets
<img src="../lectures/Lecture 2-20250316/4.png" alt="Classifying lines with original targets"/>

In [ ]:
w1, w2 = W.numpy().reshape(-1)
b = b.numpy()
x = np.linspace(-2, 4, 100)
y = -w1/w2*x + (0.5-b)/w2
# Wizualizacja predykcji w stylu "plt.scatter"
plt.figure(figsize=(8, 6))

# Rysowanie klasy 0 (fioletowy)
points1 = inputs.numpy()[predicted_classes.flatten() == 0]
plt.scatter(points1[:, 0], points1[:, 1], color='blue')

# Rysowanie klasy 1 (żółty)
points2 = inputs.numpy()[predicted_classes.flatten() == 1]
plt.scatter(points2[:, 0], points2[:, 1], color='yellow')

plt.plot(x, y, color='black')
plt.show()

plt.figure(figsize=(8, 6))

# Rysowanie klasy 0 (fioletowy) - dla oryginalnych etykiet
points1 = inputs.numpy()[targets.numpy().flatten() == 0]
plt.scatter(points1[:, 0], points1[:, 1], color='blue')

# Rysowanie klasy 1 (żółty) - dla oryginalnych etykiet
points2 = inputs.numpy()[targets.numpy().flatten() == 1]
plt.scatter(points2[:, 0], points2[:, 1], color='yellow')
plt.plot(x, y, color='black')
plt.show()


## Taks 8
Create 3 keras sequential models having the last layer with 1 unit and the sigmoid activation function. The second and third model are to have 2 layers with the first layer having the relu activation function and 2, 10 units, respectively. To compile all three models use the rmsprop optimizer, the binary_crossentropy loss function, and the accuracy metric. Fit all models using:

```python
training_inputs,
training_targets,
epochs=50,
batch_size=16,
validation_data=(val_inputs, val_targets)
```

where `inputs` and `targets` are from Task 1 and

```python
indices_permutation = np.random.permutation(len(inputs))
shuffled_inputs = inputs[indices_permutation]
shuffled_targets = targets[indices_permutation]

num_validation_samples = int(0.3 * len(inputs))
val_inputs = shuffled_inputs[:num_validation_samples]
val_targets = shuffled_targets[:num_validation_samples]
training_inputs = shuffled_inputs[num_validation_samples:]
training_targets = shuffled_targets[num_validation_samples:]
```

Evaluate all models for `val_inputs`, `val_targets`, `batch_size=16`. Which model has the smallest loss? Do predictions for all models with `val_inputs` and `batch_size=16`. Finally visualize `val_inputs` with original targets and targets predicted by all three models. 
<img src="../lectures/Lecture 2-20250316/5.png" alt="Original data and predictions"/>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

indices_permutation = np.random.permutation(len(inputs))
indices_permutation = tf.convert_to_tensor(indices_permutation, dtype=tf.int32)

shuffled_inputs = tf.gather(inputs, indices_permutation)
shuffled_targets = tf.gather(targets, indices_permutation)

num_validation_samples = int(0.3 * len(inputs))
val_inputs = shuffled_inputs[:num_validation_samples]
val_targets = shuffled_targets[:num_validation_samples]
training_inputs = shuffled_inputs[num_validation_samples:]
training_targets = shuffled_targets[num_validation_samples:]

model1 = keras.Sequential([
    layers.Dense(1, activation='sigmoid')
])
model1.compile(optimizer='rmsprop',
               loss='binary_crossentropy',
               metrics=['accuracy'])
model1.fit(training_inputs, training_targets, epochs=50, batch_size=16, validation_data=(val_inputs, val_targets))

model2 = keras.Sequential([
    layers.Dense(2, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
model2.compile(optimizer='rmsprop',
               loss='binary_crossentropy',
               metrics=['accuracy'])
model2.fit(training_inputs, training_targets, epochs=50, batch_size=16, validation_data=(val_inputs, val_targets))

model3 = keras.Sequential([
    layers.Dense(10, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
model3.compile(optimizer='rmsprop',
               loss='binary_crossentropy',
               metrics=['accuracy'])
model3.fit(training_inputs, training_targets, epochs=50, batch_size=16, validation_data=(val_inputs, val_targets))

model1_loss = model1.evaluate(val_inputs, val_targets, batch_size=16)
model2_loss = model2.evaluate(val_inputs, val_targets, batch_size=16)
model3_loss = model3.evaluate(val_inputs, val_targets, batch_size=16)
print(f"Model 1 loss: {model1_loss}")
print(f"Model 2 loss: {model2_loss}")
print(f"Model 3 loss: {model3_loss}")

predictions1 = model1.predict(val_inputs, batch_size=16)
predictions2 = model2.predict(val_inputs, batch_size=16)
predictions3 = model3.predict(val_inputs, batch_size=16)

predicted_classes1 = (predictions1 > 0.5).astype(int)
predicted_classes2 = (predictions2 > 0.5).astype(int)
predicted_classes3 = (predictions3 > 0.5).astype(int)

plt.figure(figsize=(16, 12))

plt.subplot(2, 2, 1)
plt.title("Model 1 Predictions")
points1 = val_inputs.numpy()[predicted_classes1.flatten() == 0]
points2 = val_inputs.numpy()[predicted_classes1.flatten() == 1]
plt.scatter(points1[:, 0], points1[:, 1], color='blue')
plt.scatter(points2[:, 0], points2[:, 1], color='yellow')

plt.subplot(2, 2, 2)
plt.title("Model 2 Predictions")
points1 = val_inputs.numpy()[predicted_classes2.flatten() == 0]
points2 = val_inputs.numpy()[predicted_classes2.flatten() == 1]
plt.scatter(points1[:, 0], points1[:, 1], color='blue')
plt.scatter(points2[:, 0], points2[:, 1], color='yellow')

plt.subplot(2, 2, 3)
plt.title("Model 3 Predictions")
points1 = val_inputs.numpy()[predicted_classes3.flatten() == 0]
points2 = val_inputs.numpy()[predicted_classes3.flatten() == 1]
plt.scatter(points1[:, 0], points1[:, 1], color='blue')
plt.scatter(points2[:, 0], points2[:, 1], color='yellow')

plt.subplot(2, 2, 4)
plt.title("Original Data")
points1 = val_inputs.numpy()[val_targets.numpy().flatten() == 0]
points2 = val_inputs.numpy()[val_targets.numpy().flatten() == 1]
plt.scatter(points1[:, 0], points1[:, 1], color='blue')
plt.scatter(points2[:, 0], points2[:, 1], color='yellow')

plt.show()